In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -q monai

In [2]:
import os
import sys
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader

import monai
from monai.data import Dataset, CacheDataset, list_data_collate, decollate_batch
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityd,
    RandCropByPosNegLabeld,
    RandRotate90d,
    RandShiftIntensityd,
    ToTensord,
    CropForegroundd,
    ResizeWithPadOrCropd,
    Spacingd,
    RandFlipd,
    RandScaleIntensityd,
    RandAffined,
    Rand3DElasticd,
    RandGaussianNoised,
    RandAdjustContrastd,
    Activations,
    AsDiscrete,
    SqueezeDimd,
    ToNumpyd
)
from monai.networks.nets import UNet, SegResNet
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.visualize import plot_2d_or_3d_image
from monai.utils import set_determinism
from monai.networks.utils import one_hot
from monai.config import print_config


# Set deterministic training for reproducibility
set_determinism(seed=42)

# Define paths
data_dir = "/kaggle/input/mri-segmentation/MRI_dataset"  # Update with your data path
train_output_dir = "/kaggle/working/"
os.makedirs(train_output_dir, exist_ok=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}"

Using device: cuda


In [3]:
# Define data loaders
def get_data_dicts(data_dir):
    """Create data dictionaries with paths to images and labels"""
    train_files = []
    val_files = []
    
    # Assuming your data is organized as:
    # data_dir/
    #   - raw_images/
    #       - img1.nii.gz
    #       - img2.nii.gz
    #       ...
    #   - labels/
    #       - img1_label.nii.gz
    #       - img2_label.nii.gz
    #       ...
    
    # Get image and label file paths
    all_images = sorted(glob.glob(os.path.join(data_dir, "imagesTr", "*.nii")))
    all_labels = sorted(glob.glob(os.path.join(data_dir, "labelsTr", "*.nii")))
    
    # Split into training (80%) and validation (20%)
    num_total = len(all_images)
    num_train = int(num_total * 0.8)
    
    # Create dictionaries for training data
    for img_path, label_path in zip(all_images[:num_train], all_labels[:num_train]):
        train_files.append({
            "image": img_path,
            "label": label_path
        })
    
    # Create dictionaries for validation data
    for img_path, label_path in zip(all_images[num_train:], all_labels[num_train:]):
        val_files.append({
            "image": img_path,
            "label": label_path
        })
    
    print(f"Training samples: {len(train_files)}, Validation samples: {len(val_files)}")
    return train_files, val_files

In [4]:
# Define transforms
def get_transforms():
    """Define preprocessing and augmentation transforms"""
    # Training transforms
    train_transforms = Compose([
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),  # Ensure channel first for both
        Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        ScaleIntensityd(keys=["image"]),  # Scale intensity for the RAW image
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=4,
        ),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
        RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
        RandScaleIntensityd(keys=["image"], factors=0.1, prob=0.5),
        RandShiftIntensityd(keys=["image"], offsets=0.1, prob=0.5),
        RandAffined(
            keys=["image", "label"],
            prob=0.5,
            rotate_range=(0.05, 0.05, 0.05),
            scale_range=(0.1, 0.1, 0.1),
            mode=("bilinear", "nearest"),
        ),
        Rand3DElasticd(
        keys=["image", "label"],
        prob=0.3,
        sigma_range=(5, 8),
        magnitude_range=(50, 100),
        mode=("bilinear", "nearest"),
        ),
        RandAdjustContrastd(keys=["image"], prob=0.3, gamma=(0.7, 1.5)),
        RandGaussianNoised(keys=["image"], prob=0.3, mean=0, std=0.05),
        ToTensord(keys=["image", "label"]),
    ])
    
    # Validation transforms (no augmentation)
    reg_size = (197, 233, 189)
    val_transforms = Compose([
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=reg_size),
        ScaleIntensityd(keys=["image"]),
        ToTensord(keys=["image", "label"]),
    ])
    
    return train_transforms, val_transforms

In [5]:
# Create datasets and dataloaders
def get_dataloaders(train_files, val_files, train_transforms, val_transforms, cache=False):
    """Create training and validation dataloaders"""
    # Create datasets
    if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0)
        val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0)
    else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        val_ds = Dataset(data=val_files, transform=val_transforms)
    
    # Create dataloaders
    train_loader = DataLoader(
        train_ds,
        batch_size=1,
        shuffle=True,
        num_workers=4,
        pin_memory=torch.cuda.is_available(),
        collate_fn=list_data_collate
    )
    
    val_loader = DataLoader(
        val_ds,
        batch_size=4,
        shuffle=False,
        num_workers=4,
        pin_memory=torch.cuda.is_available(),
        collate_fn=list_data_collate
    )
    
    return train_loader, val_loader

In [6]:
# Model definition
def get_model():
    """Create a SegResNet model with 1 input channel (RAW) and 3 output channels (CSF, GM, WM)"""
    model = SegResNet(
        blocks_down=[1, 2, 2, 4],
        blocks_up=[1, 1, 1],
        init_filters=16,
        in_channels=1,  # 1 channel for RAW MRI input
        out_channels=4,  # 4 channels for background, CSF, GM, WM
        dropout_prob=0.2,
    )
    
    return model

In [7]:
# Train function
def train(model, train_loader, val_loader, max_epochs=300, val_interval=5):
    """Train the model"""
    # Loss function and optimizer
    loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)  # No onehot needed since labels are already in channels
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)
    
    # Dice metric for validation
    dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
    
    # Best metric tracking
    best_metric = -1
    best_metric_epoch = -1
    
    # Initialize lists for plotting metrics
    epoch_loss_values = []
    metric_values = []
    lr_values = []
    
    # Training loop
    for epoch in range(max_epochs):
        print(f"\nEpoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0
        
        # Training steps
        train_bar = tqdm(train_loader)
        for batch_data in train_bar:
            step += 1
            inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
            
            #num_classes = 3 # (CSF, GM, WM)
            #labels = one_hot(labels, num_classes).to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            train_bar.set_description(f"Train loss: {loss.item():.4f}")
        
        # Update learning rate
        scheduler.step()
        lr_values.append(scheduler.get_last_lr()[0])
        
        # Calculate average loss for the epoch
        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"Epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        
        # Validation
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_loss = 0
                val_steps = 0
                for val_data in tqdm(val_loader, desc="Validation"):
                    val_inputs, val_labels = val_data["image"].to(device), val_data["label"].to(device)
            
                    #num_classes = 3 # (CSF, GM, WM)
                    #val_labels = one_hot(val_labels, num_classes).to(device)
                    
                    # Sliding window inference for larger volumes
                    roi_size = (96, 96, 96)
                    sw_batch_size = 4

                    val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
                    
                    # Calculate validation loss
                    val_loss += loss_function(val_outputs, val_labels).item()
                    
                    val_steps += 1
                    
                    # Calculate Dice score
                    val_outputs_list = decollate_batch(val_outputs)
                    val_labels_list = decollate_batch(val_labels)
                    val_outputs_convert = [Activations(sigmoid=True)(i) for i in val_outputs_list]
                    val_outputs_convert = [AsDiscrete(threshold=0.5)(i) for i in val_outputs_convert]
                    dice_metric(y_pred=val_outputs_convert, y=val_labels_list)
                    
                    # Plot images for visualization
                    if epoch + 1 == max_epochs and val_steps == 1:
                        # Input image (middle slice)
                        middle_slice = val_inputs.shape[4] // 2
                        plt.figure("Visualization", (18, 6))
                        plt.subplot(1, 5, 1)
                        plt.title("Input MRI")
                        plt.imshow(val_inputs[0, 0, :, :, middle_slice].cpu().numpy(), cmap="gray")
                        
                        # Ground truth labels (CSF, GM, WM)
                        plt.subplot(1, 5, 2)
                        plt.title("CSF (Ground Truth)")
                        plt.imshow(val_labels[0, 0, :, :, middle_slice].cpu().numpy(), cmap="jet")
                        
                        plt.subplot(1, 5, 3)
                        plt.title("GM (Ground Truth)")
                        plt.imshow(val_labels[0, 1, :, :, middle_slice].cpu().numpy(), cmap="jet")
                        
                        plt.subplot(1, 5, 4)
                        plt.title("WM (Ground Truth)")
                        plt.imshow(val_labels[0, 2, :, :, middle_slice].cpu().numpy(), cmap="jet")
                        
                        # Predicted segmentation (overlay)
                        pred = torch.zeros_like(val_inputs[0, 0])
                        for c in range(3):
                            pred[val_outputs[0, c] > 0.5] = c + 1
                        
                        plt.subplot(1, 5, 5)
                        plt.title("Prediction (RGB)")
                        plt.imshow(pred[:, :, middle_slice].cpu().numpy(), cmap="viridis")
                        
                        plt.tight_layout()
                        plt.savefig(os.path.join(train_output_dir, f"visualization_epoch_{epoch+1}.png"))
                        plt.close()
                
                # Calculate average validation loss and Dice scores
                avg_val_loss = val_loss / val_steps
                metric = dice_metric.aggregate().item()
                dice_metric.reset()
                
                metric_values.append(metric)
                
                # Log validation metrics
                print(f"Validation loss: {avg_val_loss:.4f}, Dice score: {metric:.4f}")
                
                # Save best model
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), os.path.join(train_output_dir, "best_model.pth"))
                    print(f"Saved new best model (Dice: {best_metric:.4f})")
    
    # Print best metric
    print(f"\nTraining completed. Best Dice: {best_metric:.4f} at epoch {best_metric_epoch}")
    
    # Plot training curves
    plt.figure("Train", (12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Dice Loss")
    x = [i + 1 for i in range(len(epoch_loss_values))]
    y = epoch_loss_values
    plt.xlabel("Epoch")
    plt.plot(x, y, label="Training Loss")
    plt.subplot(1, 2, 2)
    plt.title("Mean Dice Score")
    val_epochs = [val_interval * (i + 1) for i in range(len(metric_values))]
    plt.xlabel("Epoch")
    plt.plot(val_epochs, metric_values, label="Validation Dice")
    plt.legend()
    plt.savefig(os.path.join(train_output_dir, "training_curves.png"))
    
    # Plot learning rate
    plt.figure("Learning Rate", (6, 6))
    plt.title("Learning Rate Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Learning Rate")
    plt.plot(x, lr_values)
    plt.savefig(os.path.join(train_output_dir, "learning_rate.png"))
    
    return model, best_metric, best_metric_epoch

In [8]:
# Function to visualize class-wise Dice scores
def visualize_class_dice(model, val_loader, class_names=["CSF", "GM", "WM"]):
    """Visualize class-wise Dice scores"""
    model.eval()
    
    # Initialize class-wise Dice metrics
    dice_metrics = {class_name: DiceMetric(include_background=False, reduction="mean") 
                    for class_name in class_names}
    
    with torch.no_grad():
        for val_data in tqdm(val_loader, desc="Evaluating class-wise performance"):
            val_inputs, val_labels = val_data["image"].to(device), val_data["label"].to(device)
                
            # Sliding window inference
            roi_size = (96, 96, 96)
            sw_batch_size = 4

            val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
            
            # Process predictions
            val_outputs_list = decollate_batch(val_outputs)
            val_labels_list = decollate_batch(val_labels)
            val_outputs_convert = [Activations(sigmoid=True)(i) for i in val_outputs_list]
            val_outputs_convert = [AsDiscrete(threshold=0.5)(i) for i in val_outputs_convert]
            
            # Calculate class-wise Dice scores
            for c, class_name in enumerate(class_names):
                # Extract single class channel for both prediction and ground truth
                class_outputs = [x[:, c:c+1] for x in val_outputs_convert]
                class_labels = [x[:, c:c+1] for x in val_labels_list]
                dice_metrics[class_name](y_pred=class_outputs, y=class_labels)
    
    # Aggregate and plot class-wise dice scores
    class_dice_scores = [dice_metrics[class_name].aggregate().item() for class_name in class_names]
    
    plt.figure("Class-wise Dice Scores", (10, 6))
    plt.bar(class_names, class_dice_scores)
    plt.title("Dice Score by Brain Structure")
    plt.xlabel("Structure")
    plt.ylabel("Dice Score")
    for i, v in enumerate(class_dice_scores):
        plt.text(i, v + 0.01, f"{v:.3f}", ha='center')
    plt.ylim(0, 1.0)
    plt.tight_layout()
    plt.savefig(os.path.join(train_output_dir, "class_dice_scores.png"))
    
    return class_dice_scores

In [ ]:
# Main function to run the training pipeline
def main():
    # Get data
    train_files, val_files = get_data_dicts(data_dir)
    
    # Get transforms
    train_transforms, val_transforms = get_transforms()
    
    # Get dataloaders
    train_loader, val_loader = get_dataloaders(train_files, val_files, train_transforms, val_transforms, cache=True)
    
    # Get model
    model = get_model()  # 1 input channel (RAW), 3 output channels (CSF, GM, WM)
    model = model.to(device)
    
    # Train model
    model, best_metric, best_metric_epoch = train(model, train_loader, val_loader, max_epochs=200, val_interval=10)
    
    # Save final model
    torch.save(model.state_dict(), os.path.join(train_output_dir, "final_model.pth"))
    
    # Visualize class-wise dice scores
    print("Evaluating class-wise performance...")
    class_dice_scores = visualize_class_dice(model, val_loader, class_names=["CSF", "GM", "WM"])
    
    # Create training progress visualization
    print("Generating training progress plots...")
    
    # Example: Create a visualization of sample predictions at different training stages
    # Load the best model
    best_model = get_model().to(device)
    best_model.load_state_dict(torch.load(os.path.join(train_output_dir, "best_model.pth")))
    best_model.eval()
    
    # Visualize predictions on a validation sample
    with torch.no_grad():
        for val_data in val_loader:
            val_inputs = val_data["image"].to(device)
            val_labels = val_data["label"].to(device)
                
            roi_size = (96, 96, 96)
            sw_batch_size = 4
            val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, best_model)
            
            # Create final visualization
            middle_slice = val_inputs.shape[4] // 2
            
            # Create a figure with 3D segmentation results
            fig, axes = plt.subplots(2, 4, figsize=(20, 10))
            
            # Original MRI
            axes[0, 0].set_title("Original MRI")
            axes[0, 0].imshow(val_inputs[0, 0, :, :, middle_slice].cpu().numpy(), cmap="gray")
            
            # Ground truth segmentation - Each tissue type
            axes[0, 1].set_title("CSF (Ground Truth)")
            axes[0, 1].imshow(val_labels[0, 0, :, :, middle_slice].cpu().numpy(), cmap="jet")
            
            axes[0, 2].set_title("GM (Ground Truth)")
            axes[0, 2].imshow(val_labels[0, 1, :, :, middle_slice].cpu().numpy(), cmap="jet")
            
            axes[0, 3].set_title("WM (Ground Truth)")
            axes[0, 3].imshow(val_labels[0, 2, :, :, middle_slice].cpu().numpy(), cmap="jet")
            
            # Predicted segmentation - Each tissue type
            axes[1, 1].set_title("CSF (Predicted)")
            axes[1, 1].imshow((val_outputs[0, 0, :, :, middle_slice] > 0.5).cpu().numpy(), cmap="jet")
            
            axes[1, 2].set_title("GM (Predicted)")
            axes[1, 2].imshow((val_outputs[0, 1, :, :, middle_slice] > 0.5).cpu().numpy(), cmap="jet")
            
            axes[1, 3].set_title("WM (Predicted)")
            axes[1, 3].imshow((val_outputs[0, 2, :, :, middle_slice] > 0.5).cpu().numpy(), cmap="jet")
            
            # Combine predictions into a single color-coded image
            combined_pred = np.zeros((val_outputs.shape[2], val_outputs.shape[3]))
            for c in range(3):
                combined_pred[val_outputs[0, c, :, :, middle_slice].cpu().numpy() > 0.5] = c + 1
            
            axes[1, 0].set_title("Combined Prediction")
            axes[1, 0].imshow(combined_pred, cmap="viridis")
            
            plt.tight_layout()
            plt.savefig(os.path.join(train_output_dir, "final_segmentation_results.png"))
            plt.close()
            
            # Create an overlay visualization
            plt.figure("Segmentation Overlay", figsize=(10, 10))
            plt.imshow(val_inputs[0, 0, :, :, middle_slice].cpu().numpy(), cmap="gray")
            
            # Create an alpha mask for overlay
            mask = np.zeros((val_outputs.shape[2], val_outputs.shape[3], 4))
            colors = [(1, 0, 0, 0.5), (0, 1, 0, 0.5), (0, 0, 1, 0.5)]  # RGBA colors for CSF, GM, WM
            
            for c in range(3):
                pred_mask = val_outputs[0, c, :, :, middle_slice].cpu().numpy() > 0.5
                for i in range(4):  # RGBA channels
                    mask[pred_mask, i] = colors[c][i]
            
            plt.imshow(mask)
            plt.title("Segmentation Overlay (Red: CSF, Green: GM, Blue: WM)")
            plt.axis('off')
            plt.tight_layout()
            plt.savefig(os.path.join(train_output_dir, "segmentation_overlay.png"))
            
            break  # Just process one sample for visualization

    print(f"Training and evaluation completed. All results saved to {train_output_dir}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


Training samples: 108, Validation samples: 27


Loading dataset: 100%|██████████| 27/27 [00:06<00:00,  4.11it/s]



Epoch 1/200


Train loss: 0.6132: 100%|██████████| 108/108 [02:04<00:00,  1.15s/it]


...

Validation: 100%|██████████| 7/7 [00:29<00:00,  4.24s/it]


Validation loss: 0.1169, Dice score: 0.8567

Epoch 171/200


Train loss: 0.1047: 100%|██████████| 108/108 [02:13<00:00,  1.24s/it]


Epoch 171 average loss: 0.1119

Epoch 172/200


Train loss: 0.1213:  74%|███████▍  | 80/108 [01:35<00:28,  1.03s/it]

In [ ]:
model = get_model()  # Initialize model
model.load_state_dict(torch.load("/kaggle/working/final_model.pth", map_location=device))  # Load weights
model = model.to(device)  # Move to CUDA if available
model.eval()  # Set to evaluation mode

In [ ]:
visualize_class_dice(model, val_loader)

In [ ]:
val_loader.to(device)

In [ ]:
train_files, val_files = get_data_dicts(data_dir)
    
    # Get transforms
train_transforms, val_transforms = get_transforms()
    
    # Get dataloaders
train_loader, val_loader = get_dataloaders(train_files, val_files, train_transforms, val_transforms, cache=True)


In [ ]:
# Add code to check dimensions of your validation data
for i, item in enumerate(val_loader):
    print(f"Sample {i}, image shape: {item['image'].shape}, label shape: {item['label'].shape}")

In [ ]:
for i, item in enumerate(train_loader):
    print(f"Sample {i}, image shape: {item[0]['image'].shape}, label shape: {item[0]['label'].shape}")

In [ ]:
train_loader.shape

In [ ]:
for i, item in enumerate(train_loader):
    print(item[1]['image'])

In [ ]:
model